In [13]:
from cappy.balancer import Balancer
from cappy.config import testconfig, qc_dbpath
import qcodes as qc
from barreralabdrivers.utils.param_utils import paramp





In [7]:
## Balancing arguments
FREQ = 10000
DELTA = (150, 150)
INITIAL = (10, 10)
Cstand = 1  # pF
DRIVE = 300

In [9]:
qc.Instrument.close_all()   # in case of existing connections

station = qc.Station(config_file=str(testconfig))

dac = station.load_instrument("acdac")
li = station.load_instrument("lockin")
yoko = station.load_instrument("yoko")



Connected to: BARRERA ACDAC02 (serial:AD9106, firmware:2.00) in 3.05s
Connected to: Stanford_Research_Systems SR865A (serial:005296, firmware:V1.51) in 0.08s
Connected to: YOKOGAWA 765611 (serial:90ZC43353, firmware:1.10) in 0.03s


In [27]:
# parameter definitions 
vg = yoko.channel2.voltage
vg.label = "vg" 

vdd = yoko.channel1.voltage

dac.reset()

In [15]:
paramp(vg, -0.657) #setting Vgate to bias voltage -0.657 V as per trans_conductance_11_22_24
paramp(vdd, 0.1)  # ramping DC drive voltage to 0.1V

In [16]:
balancer = Balancer(dac.ch1, dac.ch4, dac.ch2, li, dac.frequency) # ch1 = control, ch4 = reference, ch2 = drive

In [17]:
# Initial balance
def balance_initial():
    balancer.excite(DRIVE, DRIVE, FREQ)
    null_voltages = balancer.balance(INITIAL, DELTA, null=True)
    c_meas = balancer.calculate_capacitance(Cstand)
    print(f"null voltages = {null_voltages}")
    print(f"Balance Point Capacitance: {c_meas} nF")


In [28]:
balance_initial()

Kr1, Kr2 = (np.float64(0.007625249078466667), np.float64(-0.016212630047086667))
Kc1, Kc2 = (np.float64(0.0162041944958), np.float64(0.007636597729579999))
P = 0.18143675454438546
null voltages = (np.float64(-0.09784584696232557), np.float64(-0.04398911420437912))
Balance Point Capacitance: -0.0003261528232077519 nF


In [29]:
dac.print_readable_snapshot()

acdac:
	parameter   value
--------------------------------------------------------------------------------
IDN          :	{'vendor': 'BARRERA', 'model': 'ACDAC02', 'serial': 'AD9106', ...
display_mode :	REMOTE 
 rror        :	0 - No Error
frequency    :	10000 (Hz)
timeout      :	5 (s)
acdac_ch1:
	parameter value
--------------------------------------------------------------------------------
phase   :	-24.208 (deg)
voltage :	0.10728 (mV)
acdac_ch2:
	parameter value
--------------------------------------------------------------------------------
phase   :	0 (deg)
voltage :	300 (mV)
acdac_ch3:
	parameter value
--------------------------------------------------------------------------------
phase   :	0 (deg)
voltage :	0 (mV)
acdac_ch4:
	parameter value
--------------------------------------------------------------------------------
phase   :	0 (deg)
voltage :	300 (mV)


In [ ]:
paramp(dac.ch1.voltage)
paramp(dac.ch2.voltage)
paramp(vg)
paramp(vdd)

acdac:
	parameter   value
--------------------------------------------------------------------------------
IDN          :	{'vendor': 'BARRERA', 'model': 'ACDAC02', 'serial': 'AD9106', ...
display_mode :	REMOTE 
 rror        :	0 - No Error
frequency    :	10000 (Hz)
timeout      :	5 (s)
acdac_ch1:
	parameter value
--------------------------------------------------------------------------------
phase   :	116.02 (deg)
voltage :	168.44 (mV)
acdac_ch2:
	parameter value
--------------------------------------------------------------------------------
phase   :	0 (deg)
voltage :	300 (mV)
acdac_ch3:
	parameter value
--------------------------------------------------------------------------------
phase   :	0 (deg)
voltage :	0 (mV)
acdac_ch4:
	parameter value
--------------------------------------------------------------------------------
phase   :	0 (deg)
voltage :	300 (mV)
